### PERCEPTRON - QUESTION 1

\begin{array}{l}g \in \partial f_{k}(x) \\ \Rightarrow f_{k}(z) \geq f_{k}(x)+g^{\top}(z-x)\end{array}

\begin{aligned} f(z) &=\max _{i=1, \ldots, m}\left\{f_{i}(z)\right\} \\ & \geq f_{k}(z) \\ & \geqslant f_{k}(x)+g^{\top}(z-x) \\ &=f(x)+g^{\top}(z-x) \\ \Rightarrow g & \in \partial f(x) \end{aligned}

### PERCEPTRON - QUESTION 2

$$g=\left\{\begin{array}{cl}-y x & \text { if } y w^{\top} x<1 \\ 0 & \text { otherwise }\end{array}\right.$$

### PERCEPTRON - QUESTION 3

\begin{array}{l}\qquad\left\{x | w^{\top} x\right\} \text { is a separating hyperplane for } D \\ \Rightarrow y_{i} w^{\top} x_{i}>0 \quad \forall \leqslant i \leqslant n \\ \Rightarrow \operatorname{sgn}\left(w^{\top} x_{i}\right)=\operatorname{sgn}\left(y_{i}\right) \\ \Rightarrow-y_{i} \operatorname{sgn}\left(w^{\top} x_{i}\right)=-y_{i} \operatorname{sgn}\left(y_{i}\right) \leq 0 \\ \text { Average perceptron loss } = \frac{1}{n} \sum_{i=1}^{n} l(\hat{y}, y) \\ =\frac{1}{n} \sum_{i=1}^{n} \max \{0,-\hat{y} y\} \\ =\frac{1}{n} \sum_{i=1}^{n} \max \left\{0,-y_{i} \operatorname{sgn}\left(w^{\top} x_{i}\right)\right\} \\ =0\end{array}

### PERCEPTRON - QUESTION 4

The weight update step of the SSGD algorithm is given by:

$$
\begin{array}{l}w_{(t+1)}=w_{(t)}-\eta g \\ \text { Choose } \eta=1 \text { and } g=\left\{\begin{array}{cl}-y_{i} x_{i} & \text { if } y_{i} w_{(t)}^{\top} x_{i}<1 \\ 0 & \text { othenwise }\end{array}\right.\end{array}
$$

$$
\begin{array}{l}\text { Then the SSGD update step can be written as the following conditional statement } \\ \text { if }\left(y_{i} w_{(t)}^{\top} x_{i}<1\right) \\ \qquad \begin{array}{l}w_{(t+1)}=w_{(t)}+y_{i} x_{i} \\ \text { else } \\ w_{(t+1)}=w_{(t)}\end{array} \\ \text { This is exactly the same condition used in the Perceptron Algorithm }\end{array}
$$

### PERCEPTRON - QUESTION 5

This follows by induction on k in the Perceptron Algorithm. <br>
Let P(t) be the proposition that the output of the perceptron algorithm is a linear combination of input points after t iterations. <br>
P(1) is true since 
\begin{aligned} w^{(1)} &=w^{(0)}+y_{1} x_{1} \\ &=y_{1} x_{1} \end{aligned}
or remains unchanged. <br>
$w$ is initialized by a zero vector. <br>

Assume P(k) is true i.e. $w^{(k)}$ is L.C. of $x_i$ <br>
In the case where $y_{i} x_{i}^{\top} w^{(k)}>0$, w stays unchanged. <br>
In the case where $y_{i} x_{i}^{\top} w^{(k)} \leq 0$, $$w^{(k+1)}=w^{(k)}+y_{i} x_{i}$$ <br>
Since $w^{(k)}$ is a L.C., $w^{(k+1)}$ is also a L.C. of $x_i$. <br>
Therefore, P(k+1) is true. <br>

The result follows by principle of mathematical induction.


### SPARSE REPRESENTATION - QUESTION 1 and 2
(implementation for QUESTION 2 has been baked into QUESTION 1 to make it more optimized)

In [12]:
import math
import random
import os
import time

from collections import Counter

import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
from copy import copy
from scipy.optimize import minimize

from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.metrics import confusion_matrix



In [2]:

DATA_DIR = '../data/svm/data/'
POS_DATA_DIR = DATA_DIR + 'pos/'
NEG_DATA_DIR = DATA_DIR + 'neg/'
TRAIN_SPLIT_COUNT = 1500


In [3]:

# read review file into a list of strings
# each string = a token from the review
def read_file(filepath):
    f = open(filepath)
    lines = f.read().split(' ')
    symbols = '${}()[].,:;+-*/&|<>=~" '
    words = map(lambda Element: Element.translate(str.maketrans("", "", symbols)).strip(), lines)
    words = filter(None, words)
    return Counter(list(words))

# return list of file names in a directory
# skips directories
def list_dir_files(dirpath):
    file_names = []
    children = os.listdir(dirpath)

    for child in children:
        if os.path.isfile(os.path.join(dirpath, child)):
            file_names.append(child)
    
    return file_names

# read all files in the directory
def read_dir(dirpath):
    file_contents = []
    file_names = list_dir_files(dirpath)

    for file_name in file_names:
        filepath = os.path.join(dirpath, file_name)
        file_content = read_file(filepath)
        file_contents.append(file_content)

    return file_contents

In [4]:
pos_reviews = read_dir(POS_DATA_DIR)
neg_reviews = read_dir(NEG_DATA_DIR)

NUM_POS_REVIEWS = len(pos_reviews)
NUM_NEG_REVIEWS = len(neg_reviews)

pos_review_points = list(zip(pos_reviews, [1] * NUM_POS_REVIEWS, range(1, NUM_POS_REVIEWS + 1)))
neg_review_points = list(zip(neg_reviews, [-1] * NUM_NEG_REVIEWS, range(1, NUM_NEG_REVIEWS + 1)))

all_points = pos_review_points.copy()
all_points.extend(neg_review_points)
random.shuffle(all_points)

train = all_points[:TRAIN_SPLIT_COUNT]
test = all_points[TRAIN_SPLIT_COUNT:]

train_unzip = list(zip(*train))
train_X = list(train_unzip[0])
train_Y = list(train_unzip[1])
train_indices = list(train_unzip[2])

test_unzip = list(zip(*test))
test_X = list(test_unzip[0])
test_Y = list(test_unzip[1])
test_indices = list(test_unzip[2])


### SVM with PEGASOS - QUESTION 1

$$
\nabla_{\omega} J_{i}(\omega)=\left\{\begin{array}{lll}\lambda w-y_{i} x_{i} & \text { if } y_{i} w^{\top} x_{i}<1 \\ \lambda \omega & \text { if } y_{i} w^{\top} x_{i}>1 \\ \text { undefined } & \text { if } y_{i} w^{\top} x_{i}=1\end{array}\right.
$$

### SVM with PEGASOS - QUESTION 2

$$
\partial J_{i}(\omega)=\partial\left(\frac{\lambda}{2}\left\|_{w}\right\|^{2}\right)+\partial\left(\max \left\{0,1-y_{i} w^{\top} x_{i}\right\}\right) \\
g=\left\{\begin{array}{ll}\lambda w-y_{i} x & \text { if } y_{i} w^{\top} x_{i}<1 \\ \lambda{w} & \text { otherwise }\end{array}\right.
$$

Using results from PERCEPTRON - QUESTION 2, subdifferential of sum of convex functions

### SVM with PEGASOS - QUESTION 3

The subgradient update expression is:

$$
\begin{array}{l}w^{(t+1)}=w^{(t)}-\eta g \\ g=\left\{\begin{array}{ll}\lambda w^{(t)}-y_{i} x_{i} & \text { if } y_{i} w^{\top} x_{i}<1 \\ \lambda w^{(t)} & \text { otherwise }\end{array}\right.\end{array} \\

w^{(t+1)}=\left\{\begin{array}{ll}w^{(t)}-\eta\left(\lambda w^{(t)}-y ; x_{i}\right) & \text { if } y_{i} w^{\top} x_{i}<1 \\ w^{(t)}-\eta \lambda w^{(t)} & \text { otherwise }\end{array}\right. \\

w^{(t+1)}=\left\{\begin{array}{ll}(1-\eta \lambda) w^{(t)}+\eta y_{i} x_{i} & \text { if } y_{i} w^{T} x_{i}<1 \\ (1-\eta \lambda) w^{(t)} & \text { otherwise }\end{array}\right.
$$

This corresponds exactly to the conditional statement in the pseudocode.

### SVM with PEGASOS - QUESTION 4

In [39]:
"""
Dot product between sparse representation of dictionaries/counters
"""
def sparse_dot(d1, d2):
    if len(d1) < len(d2):
        return sparse_dot(d2, d1)
    
    return sum(d1[k] * v for k, v in d2.items())

"""
Inplace linear combination update for d1 += z * d2
"""
def sparse_lc(d1, z, d2):
    for k, v in d2.items():
        d1[k] += z * v

"""
Sign function impl
"""
def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    return 0

def hinge_loss(X, y, w):
    m = len(X)
    loss = 0

    for j in range(m):
        loss += max(0, 1 - y[j] * sparse_dot(w, X[j]))
    
    return loss / m


In [36]:

class Pegasos():
    def __init__(self, l2_reg = 1):
        self.w = Counter()
        self.l2_reg = l2_reg
    
    def fit(self, X, y, max_epochs = 1000):
        epoch = 0
        m = len(X)
        t = 1
        while epoch < max_epochs:
            for j in range(m):
                x_j = X[j]
                y_j = y[j]

                t += 1
                step_size = 1 / (t * self.l2_reg)
                
                if y_j * sparse_dot(self.w, x_j) < 1:
                    sparse_lc(self.w, -step_size * self.l2_reg, self.w)
                    sparse_lc(self.w, step_size * y_j, x_j)
                else:
                    sparse_lc(self.w, -step_size * self.l2_reg, self.w)
            
                
            epoch += 1
        
        return self.w, epoch
    
    def predict(self, X, y = None):
        return sparse_dot(X, self.w)
    
    def score(self, X, y):
        num_misclass = 0
        m = len(X)
        for j in range(m):
            if sign(sparse_dot(X[j], self.w)) != y[j]:
                num_misclass += 1
        
        return num_misclass / m


### SVM with PEGASOS - QUESTION 5 and 7
(implementation of QUESTION 7 is baked into the class implementation for Pegasos) <br>

\begin{array}{l}\qquad W_{(t+1)}=W_{(t)}+\frac{1}{s_{(t+1)}} \eta_{t} y_{j} x_{j} \\ \Leftrightarrow \quad s_{(t+1)} W_{(t+1)}=s_{(t+1)} W_{(t)}+\eta_{t} y_{j} x_{j} \\ \Leftrightarrow \quad w_{(t+1)}=\left(1-\eta_{t} \lambda\right) s_{(t)} W_{(t)}+\eta_{t} y_{j} x_{j} \\ \Leftrightarrow w_{(t+1)}=\left(1-\eta_{t} \lambda\right) w_{(t)}+\eta_{t} y_{j}{x_j}\end{array} <br>

Therefore, both approaches are equivalent as long as $s_{(t+1)} \neq 0$

In [52]:
class Pegasos_opt():
    def __init__(self, l2_reg = 1):
        self.w = Counter()
        self.s = 1
        self.l2_reg = l2_reg
    
    def fit(self, X, y, max_epochs = 1000, min_obj_decrease=1e-8):
        epoch = 0
        m = len(X)
        t = 1
        s_ = 1
        obj_decrease = min_obj_decrease + 1.
        obj_val = hinge_loss(X, y, self.w)

        while obj_decrease > min_obj_decrease and epoch < max_epochs:
            obj_old = obj_val
            for j in range(m):
                x_j = X[j]
                y_j = y[j]

                t += 1
                step_size = 1 / (t * self.l2_reg)

                s_ = (1 - step_size * self.l2_reg) * s_

                if s_ == 0:
                    s_ = 1
                    self.w = Counter()
                else:
                    if y_j * sparse_dot(self.w, x_j) < 1:
                        sparse_lc(self.w, step_size * y_j/s_, x_j)
            
            obj_val = hinge_loss(X, y, self.w)
            obj_decrease = abs(obj_old - obj_val)
            epoch += 1
        
        sparse_lc(self.w, s_ - 1, self.w)

        return self.w, epoch
    
    def predict(self, X, y = None):
        return sparse_dot(X, self.w)
    
    def score(self, X, y):
        num_misclass = 0
        m = len(X)

        for j in range(m):
            if sign(sparse_dot(X[j], self.w)) != y[j]:
                num_misclass += 1
        
        return num_misclass / m

### SVM with PEGASOS - QUESTION 6

In [17]:

for impl in [Pegasos(1), Pegasos_opt(1)]:
    print('Running ' + str(type(impl)))
    start = time.time()
    w, e = impl.fit(train_X, train_Y, max_epochs = 2)
    end = time.time()
    
    print('Time taken = ' + str(end - start))
    print()
    for x in w.most_common(20):
        print(x)
    print()
    print()


Running <class '__main__.Pegasos'>
Time taken = 26.074629068374634

('will', 0.06564478507164302)
('and', 0.06564478507164277)
('most', 0.06231256247917362)
('life', 0.06231256247917344)
('as', 0.05764745084971683)
('very', 0.055981339553482154)
('well', 0.05431522825724774)
('is', 0.05398200599800054)
('world', 0.04865044985005)
('war', 0.044318560479840056)
('also', 0.04431856047984003)
('many', 0.043318893702099265)
('quite', 0.04298567144285229)
('their', 0.04165278240586479)
('best', 0.0393202265911364)
('both', 0.03832055981339565)
('love', 0.03765411529490172)
('we', 0.03498833722092639)
('way', 0.032989003665444945)
('with', 0.03265578140619793)


Running <class '__main__.Pegasos_opt'>
Time taken = 0.3313291072845459

('most', 0.043652115961350546)
('well', 0.037987337554142186)
('very', 0.03498833722092343)
('great', 0.0339886704431791)
('also', 0.03365544818393573)
('world', 0.03198933688770467)
('life', 0.031322892369203714)
('quite', 0.030656447850716972)
('you', 0.02932355

### SVM with PEGASOS - QUESTION 8

In [54]:
def get_top_dict_values(dict_, k, order = 0):
    """
    Sorts a dict and returns the top k elems as a list of tuples (ordered by values)
    order = 0 for ascending, order = 1 for descending
    """
    
    # map the order from {0, 1} to {-1, 1}
    order = order * 2 - 1
    
    d = {k:v for k, v in sorted(dict_.items(), key = lambda item : -order*item[1])}
    ret_list = []
    for x in list(d)[0:(k + 1)]:
        ret_list.append((x, d[x]))
    
    return ret_list

def print_top_dict_values(dict_, k, order = 0):
    """
    Prints the top k elems of the dictionary (ordered by values)
    order = 0 for ascending, order = 1 for descending
    """
    
    for x in get_top_dict_values(dict_, k, order):
        print("{}\t{}".format(x[1], x[0]))

In [61]:
# lambda_list = [10**-3, 10**-2, 0.1, 0.6, 0.8, 1, 1.05, 1.08, 1.2, 1.5, 1.8, 2, 2.2]
lambda_list = [1, 1.005, 1.01, 1.03, 1.05]
config_val_loss_map = {}

for lamba_reg in lambda_list:
    impl = Pegasos_opt(lamba_reg)
    w, e = impl.fit(train_X, train_Y, max_epochs = 500)
    loss = impl.score(test_X, test_Y)
    config_val_loss_map[(lamba_reg, e)] = loss


In [62]:
print_top_dict_values(config_val_loss_map, len(lambda_list), order = 0)

0.16	(1, 39)
0.16	(1.005, 39)
0.16	(1.01, 39)
0.16	(1.03, 39)
0.16	(1.05, 39)


The best validation loss is observed for $\lambda \in [1, 1.05]$

### KERNELS - QUESTION 1

Consider $\phi$ as the binary vectorizer. In terms of the sparse representation, $\phi(x)$ is a dictionary where a word appears in the dictionary if and only if it appears in the document and the value against is key is always 1. <br>
This fits well into the sparse inner product we have used the Pegasos implementation. <br>
Keeping the same definition of the inner product, we see that this featurization counts the number of common words in two sparse vectors. <br>

$$
\begin{array}{lll}\phi\left(x_{1}\right)= & \sum \alpha_{i} w_{1 i} & , \alpha_{i} \in\{0,1\} \\ \phi\left(x_{2}\right)= & \sum \beta_{i} w_{2 i} & ; \beta_{i} \in\{0,1\}\end{array} \\

\phi\left(x_{1}\right)^{\top} \phi\left(x_{2}\right)=\sum \alpha_{i} \beta_{i}=\sum_{\alpha_{i} \neq 0 \atop \beta_{i} \neq 0} 1
$$

The inner product thus counts the number of common words in both documents.

$$
\phi\left(x_{1}\right)^{\top} \phi\left(x_{2}\right) = k(x_1, x_2)
$$

Therefore, k is a kernel.

### KERNELS - QUESTION 2

$$
\begin{array}{l}k(x, z)=x^{\top} z \\ f(x)=1 /\|x\|_{2} \\ \Rightarrow k(x, x)=x^{\top} x \text { is a kernel } \\ \Rightarrow f(x) f(x)=k(x, x)=\frac{x^{\top} x}{\|x\|_{2}^{2}}=1 \text { is a kernel }(1)\end{array} \\

\begin{aligned} & k(x, 2) \text { is a kernel } \\ \Rightarrow & f(x) f(z) k(x, z)=\frac{x^{\top} z}{\|x\|_{2}\|z\|_{2}} \quad \text { is a kernel }(2) \end{aligned} \\

\begin{array}{l}\quad \text{By} \quad 1,2 \\ f(x) f(x) k(x, x)+f(x) f(z) k(x, z)=1+\frac{x^{\top} z}{\|x\|_{2}\|z\|_{2}} \text { is a kernel } \\ \Rightarrow\left(1+\frac{x^{\top} z}{\|x\|_{2}\|z\|_{2}}\right)\left(1+\frac{x^{\top} z}{\|x\|_{2}\|z\|_{2}}\right)\left(1+\frac{x^{\top} z}{\|x\|_{2}\|z\|_{2}}\right) \text { is a kernel} \\ \Rightarrow \quad\left(1+\frac{x^{\top} z}{\|x\|_{2}\|z\|_{2}}\right)^{3} \text { is a kernel }\end{array}
$$

### KERNEL PEGASOS - QUESTION 1

$$
\begin{aligned} y_{j}\left\langle w^{(t)}, x_{j}\right\rangle &=y_{j}\left\langle x_{j}, \sum_{i=1}^{n} \alpha_{i}^{(t)} x_{i}\right\rangle \\ &=y_{j} \sum_{i=1}^{n} \alpha_{i}^{(t)}\left\langle x_{j}, x_{i}\right\rangle \\ &=y_{j} K_{j} \alpha^{(t)} \end{aligned}
$$

### KERNEL PEGASOS - QUESTION 2

$$
\begin{aligned} w^{(t+1)}=&\left(1-\frac{1}{t}\right) w^{(t)} \\ \sum_{i=1}^{n} \alpha_{i}^{(t+1)} x_{i} &=\left(1-\frac{1}{t}\right) \sum_{i=1}^{n} \alpha_{i}^{(t)} x_{i} \\ &=\sum_{i=1}^{n}\left(1-\frac{1}{t}\right) \alpha_{i}^{(t)} x_{i} \\ \alpha^{(t+1)}=&\left(1-\frac{1}{t}\right) \alpha^{(t)} \end{aligned}
$$

### KERNEL PEGASOS - QUESTION 3

$$
\begin{aligned} w^{(t+1)} &=\left(1-\frac{1}{t}\right) w^{(t)}+\eta y_{j} x_{j} \\ \sum_{i=1}^{n} \alpha_{i}^{(t+1)} x_{i} &=\left(1-\frac{1}{t}\right) \sum_{i=1}^{n} \alpha_{i}^{(t)} x_{i}+\sum_{i=1}^{n} \eta y_{j} \delta_{i j} x_{i} \quad\left(\delta_{i j}=\text { kronecker delta }\right) \\ &=\sum_{i=1}^{n}\left[\left(1-\frac{1}{t}\right) \alpha_{i}^{(t)}+\eta y_{j} \delta_{i j}\right] x_{i} \\ \alpha^{(t+1)} &=\left(1-\frac{1}{t}\right) \alpha^{(t)}+\eta y_{j} \hat{e}_{j} \quad\left(\hat{e}_{j}=\text { canonical basis vector in direction j }\right) \end{aligned}
$$

```
input lambda > 0.
Choose alpha_1 = 0, t = 0
while termination condition is not met:
    for j = 1, 2, ..., m:
        t = t + 1
        eta = 1 / (lambda * t)

        if y_j * dot(K_j, alpha_{t}) < 1:
            alpha_{t+1} = (1 - 1/t) * alpha_{t} + eta * y_j * e_j
        else:
            alpha_{t+1} = (1 - 1/t) * alpha_{t}
```